In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf

# Import and inspect data

## Import

In [ ]:
def import_data():
    """Import training and testing data, as data and labels"""
    
    # Import in pandas (np.genfromtxt is too slow)
    tr_pd = pd.read_csv('../week_2/data_train.csv', header=None)
    tst_pd = pd.read_csv('../week_2/data_test.csv', header=None)
    
    # Split data and labels
    tr_data = np.array(tr_pd.loc[:, 1:])
    tr_labels = np.array(tr_pd.loc[:, 0])
    tst_data = np.array(tst_pd.loc[:, 1:])
    tst_labels = np.array(tst_pd.loc[:, 0])
    
    return tr_data, tst_data, tr_labels, tst_labels

In [ ]:
g_tr_data, g_tst_data, g_tr_labels, g_tst_labels = import_data()
g_tr_data = g_tr_data/255
g_tst_data = g_tst_data/255

## Inspect

In [ ]:
def inspect_number(data, img_nums, savefig=False):
    """Display image. img_nums should be entered as list."""
    
    num_images = len(img_nums)
    # Reshape datasets into pixel array
    data_rs = np.reshape(data, newshape=(data.shape[0], 28, 28))
    
    figsize = (10, 1)
    fig, axes = plt.subplots(1, num_images, figsize=figsize)
    for k in range(num_images):
        ax = axes[img_nums[k]]
        ax.imshow(data_rs[img_nums[k]], cmap='Greys')
        ax.set_axis_off()
    if savefig:
        plt.savefig('number_output_pic.pdf')
    plt.show()

In [ ]:
inspect_number(g_tr_data, np.arange(10), savefig=True)

# Build VAE in Tensorflow

In [ ]:
class AutoEncoder():
    """Autoencoder. For most examples, self.inp=self.tgt, but these are considered 
    independent variables to make extensions more straightforward."""
    
    def __init__(self, num_z, learn_param):
        self.batch_size = 100
        self.num_z = num_z    # Number of dimensions of latent space
        self.learn_param = learn_param
        
        self.inp = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='inputs')
        self.tgt = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='targets')
        self.sess = tf.Session()
        self._build_graph()
        self.sess.run(tf.initializers.global_variables())
    
    def _build_graph(self):
        """Build neural network"""
        initializer = tf.glorot_uniform_initializer()
        
        # Encoder: convolutional neural network
        enc_h = tf.reshape(self.inp, shape=[-1, 28, 28, 1])    
        enc_h = tf.layers.conv2d(enc_h, 16, kernel_size=[5, 5], strides=[2, 2], padding='same', 
                             kernel_initializer=initializer, activation=tf.nn.relu, 
                             name='enc_conv1')    # 28x28 -> 14x14x16
        print(enc_h)
        enc_h = tf.layers.conv2d(enc_h, 32, kernel_size=[5, 5], strides=[2, 2], padding='same',
                             kernel_initializer=initializer, activation=tf.nn.relu, 
                             name='enc_conv2')    # 14x14x16 -> 7x7x32
        print(enc_h)
        enc_h = tf.reshape(enc_h, [-1, 7*7*32])

        # Bottleneck with reparametrisation trick
        self.mu = tf.layers.dense(enc_h, self.num_z, kernel_initializer=initializer,
                                  activation=None, name='mu')
        print(self.mu)
        self.log_sigma = tf.layers.dense(enc_h, self.num_z, kernel_initializer=initializer,
                                         activation=None, name='log_sigma')
        print(self.log_sigma)
        self.z = tf.add(self.mu, 
                        tf.multiply(tf.exp(0.5 * self.log_sigma), 
                                    tf.random.normal(shape=[tf.shape(self.mu)[0], 
                                                            self.num_z]),
                        name='z'))
        print(self.z)

        # Decoder: deconvolutional neural network
        dec_h = tf.layers.dense(self.z, 7*7*32, kernel_initializer=initializer, 
                            activation=tf.nn.relu, name='dec_dense1')
        print(dec_h)
        dec_h = tf.reshape(dec_h, [-1, 7, 7, 32])
        print(dec_h)
        dec_h = tf.layers.conv2d_transpose(dec_h, 16, kernel_size=[5, 5], strides=[2, 2], padding='same', 
                             kernel_initializer=initializer, activation=tf.nn.relu, 
                             name='dec_deconv1')    # 7x7x32 -> 14x14x16
        print(dec_h)
        dec_h = tf.layers.conv2d_transpose(dec_h, 1, kernel_size=[5, 5], strides=[2, 2], padding='same', 
                             kernel_initializer=initializer, activation=tf.nn.sigmoid, 
                             name='dec_deconv2')    # 14x14x16 -> 28x28x1
        print(dec_h)
        self.out = tf.reshape(dec_h, [-1, 784])
    
        # Optimizer: sum of generation loss and KL divergence
        self.loss_gen = -tf.reduce_sum(self.tgt * tf.log(1e-10 + self.out) + \
                                       (1 - self.tgt) * tf.log(1e-10 + 1 - self.out), 
                           axis=1)
        self.loss_KL = \
            0.5 * tf.reduce_sum(tf.square(self.mu) + tf.square(tf.exp(self.log_sigma)) - \
                                tf.log(tf.square(tf.exp(self.log_sigma))) - 1, axis=1)
        self.cost = tf.reduce_mean(tf.add(self.loss_gen, self.loss_KL))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=self.learn_param).minimize(self.cost)
        
    def make_minibatch(self, data_inp, data_tgt, batch_num):
        """Form a minibatch from the data"""
        llim = batch_num * self.batch_size
        rlim = (batch_num + 1) * self.batch_size
        return data_inp[llim:rlim], data_tgt[llim:rlim]
    
    def evaluate_AE_outputs(self, data_inp, data_tgt):
        """Run the autoencoder on some data and evaluate the mean squared error"""
        feed_dict = {self.inp: data_inp, self.tgt: data_tgt}
        z, outputs, cost = self.sess.run([self.z, self.out, self.cost], feed_dict=feed_dict)
        return z, outputs, cost
    
    def test_random(self, data_inp):
        feed_dict = {self.inp: data_inp}
        mu, log_sigma, z = self.sess.run([self.mu, self.log_sigma, self.z], feed_dict=feed_dict)
        return mu, log_sigma, z

    def estimate_plots(self, data_inp, img_nums, savefig=False):
        """Plot the autoencoder estimates over some images"""
        self.run_size = data_inp.shape[0]
        estimates = self.evaluate_AE_outputs(data_inp[img_nums], data_inp[img_nums])[1]
        # print(np.round(estimates[0, :10], 20))   # For debugging
        inspect_number(estimates, img_nums, savefig=savefig)
        
    def plot_z(self, data_z, data_labels, savefig=False):
        """Plot z values in 2-space (if z-space is 2-dimensional)"""
        fig, ax = plt.subplots()
        for label in range(10):
            data = data_z[data_labels == label]
            ax.scatter(data[:, 0], data[:, 1])
        ax.set_xlabel('z1')
        ax.set_ylabel('z2')
        plt.axis('equal')
        if savefig:
            plt.savefig('z_output_pic.pdf')
        plt.show()
    
    def train_iteration(self, data_inp, data_tgt):
        """Do one training iteration over the input data"""
        self.run_size = data_inp.shape[0]
        feed_dict = {self.inp: data_inp, self.tgt: data_tgt}
        _, out = self.sess.run([self.optimizer, self.out], feed_dict=feed_dict)
        
    def train_full(self, tr_inp, tr_tgt, tst_inp, tst_tgt, num_epochs, 
                   info=True, infoplots=False):
        """Train neural network and keep track of progress by (cross) validating"""
        
        # batch_size must divide num_tr_points
        num_tr_pts = tr_inp.shape[0]
        num_tst_pts = tst_inp.shape[0]
        num_batches = int(round(num_tr_pts / self.batch_size)) 
        
        # Train the MLP and keep track of the errors across training and testing datasets
        batch_nums, epochs, tr_costs, tst_costs = \
                np.array([]), np.array([]), np.array([]), np.array([])
        for epoch in range(num_epochs):
            for batch_num in range(num_batches):
                # Input proportion correctly identified across training, testing dataset
                if batch_num % 20 == 0:
                    tr_z, tr_out, tr_cost = self.evaluate_AE_outputs(tr_inp, tr_tgt)
                    tst_z, tst_out, tst_cost = self.evaluate_AE_outputs(tst_inp, tst_tgt)
#                    self.plot_z(tr_z, g_tr_labels)
                    if info is True:
                        print('Cost after {} batches: Tr {}, Tst {}'.format(
                            batch_num, tr_cost, tst_cost))
                    if infoplots is True: 
                        self.estimate_plots(g_tr_data, np.arange(10))
                    epochs = np.append(epochs, epoch)
                    batch_nums = np.append(batch_nums, batch_num)
                    tr_costs = np.append(tr_costs, tr_cost)
                    tst_costs = np.append(tst_costs, tst_cost)
                
                # Do the training over a minibatch
                mb_tr_inp, mb_tr_tgt = \
                    self.make_minibatch(tr_inp, tr_tgt, batch_num=batch_num) 
                self.train_iteration(mb_tr_inp, mb_tr_tgt)    # Single training iteration
                
        if infoplots is True:
            self.estimate_plots(g_tr_data, np.arange(10), savefig=False)
#            self.plot_z(tr_z, g_tr_labels, savefig=False)
                
        costs = pd.DataFrame(np.vstack((epochs, batch_nums, tr_costs, tst_costs)).T, 
                             columns = ['epoch', 'batch', 'train', 'test'])
        return costs

In [ ]:
tf.reset_default_graph()
model = AutoEncoder(num_z=10, learn_param=0.01)
costs = model.train_full(g_tr_data, g_tr_data, g_tst_data, g_tst_data,
                         num_epochs=5, info=False, infoplots=True)

# Train model and evaluate